In [31]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan
import json

In [32]:
#define the relevant path to the current batch you wish to parse
gap_name='pilot_24_hours' #dont change this as this parsing sciprt only handles the same day batches.  

project_name='multicategory'
experiment_name='experiment 1'
batch_name='batch 3'

PATH_TO_BATCH=path.Path.cwd().parent.parent.parent / 'projects' / project_name / experiment_name / 'collected data' / gap_name / batch_name
PATH_TO_BATCH_DATA = PATH_TO_BATCH / 'data'

if PATH_TO_BATCH.exists():
    print('batch folder found at:\n' ,PATH_TO_BATCH)
else: 
    print('path to batch is non existent:\n',PATH_TO_BATCH)

batch folder found at:
 c:\Users\User\Desktop\visual_memory\visual_memory_representation_final\projects\multicategory\experiment 1\collected data\pilot_24_hours\batch 3


In [33]:
qualification_name_for_testin='UPDATE-completed encoding multicategory memory test'  #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined
qualification_name_for_entire_experiment='UPDATE-completed multicategory memory test'

In [34]:
#this section loads the shelf dict in the state it is after finishing the testing session for this batch. 
shelf_dict_after_encoding_name=PATH_TO_BATCH / 'shelf after encoding session closed.txt' #define the name of the relevant shelf for this stage
with open(shelf_dict_after_encoding_name) as f:
    data = f.read()
shelf_dict = json.loads(data)

In [35]:
#this section reads all the relevant files: 
if (PATH_TO_BATCH / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(PATH_TO_BATCH / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (PATH_TO_BATCH / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(PATH_TO_BATCH / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (PATH_TO_BATCH / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(PATH_TO_BATCH / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [36]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [37]:
def process_worker_results(PATH_TO_BATCH_DATA,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]

    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest','demo_bar_2.response']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target','correct','test_encoding_response.rt','key_resp_end.keys']
    
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [38]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_BATCH_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]
#print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT','demo_rating_var'])
for subject_csv in all_filenames:
    subject_dict=process_worker_results(PATH_TO_BATCH_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']
    cur_sub_demo = subject_dict['demo_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    # demo rating var : 
    rating_std = cur_sub_demo['demo_bar_2.response'].std()
    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT,'demo_rating_var':rating_std},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [39]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1)& (qualification_for_test_df['demo_rating_var']>0),:].copy()
print('qualified participants')
qualified_df

qualified participants


,workerID,arrow_acc,mean_arrow_RT,demo_rating_var,in_encoding_workers_list
A105M4JOZSR7E6,A105M4JOZSR7E6,0.6,1.231933,1.211060,1.0
A151XA53WYXVD7,A151XA53WYXVD7,1.0,1.084400,0.516398,1.0
A1969Q0R4Y0E3J,A1969Q0R4Y0E3J,1.0,0.507540,1.940790,1.0
A1DDWUJ5R357SW,A1DDWUJ5R357SW,1.0,1.129760,0.894427,1.0
A1HLQV5HX3WUK4,A1HLQV5HX3WUK4,1.0,1.624940,0.547723,1.0
A1IMLTXT2RBEFO,A1IMLTXT2RBEFO,1.0,1.242800,0.752773,1.0
A1SL10V8CDNI9H,A1SL10V8CDNI9H,1.0,1.055400,0.752773,1.0
A1VJ4ZOXP9PUCP,A1VJ4ZOXP9PUCP,1.0,0.876480,0.752773,1.0
A1Y8FC2FCI0RZ5,A1Y8FC2FCI0RZ5,1.0,1.013040,0.752773,1.0
A20ZFQ93DF0WA,A20ZFQ93DF0WA,1.0,1.147200,0.752773,1.0


In [40]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0)&(qualification_for_test_df['demo_rating_var']>0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,demo_rating_var,in_encoding_workers_list
A1FN28N3CFL552,A1FN28N3CFL552,1.0,1.107800,1.095445,0.0
A1T8VILAKHKGR1,A1T8VILAKHKGR1,0.6,1.165533,0.816497,0.0
A2646YGRS0G84,A2646YGRS0G84,1.0,1.035800,1.032796,0.0
A2F2JW30D1348R,A2F2JW30D1348R,1.0,1.194380,1.378405,0.0
A2F8NXRMUUV7J5,A2F8NXRMUUV7J5,1.0,1.085880,2.338090,0.0
A2M5S1RXUCVE7L,A2M5S1RXUCVE7L,1.0,1.141600,1.095445,0.0
AO6JZCCNF0LFS,AO6JZCCNF0LFS,1.0,0.565000,1.048809,0.0


In [41]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,demo_rating_var,in_encoding_workers_list
A16BUREYCTMSOC,A16BUREYCTMSOC,0.0,NaN,0.408248,1.0
A18GEAFI5MMW02,A18GEAFI5MMW02,0.0,NaN,0.000000,1.0
A1B8OVQXUY3WQ6,A1B8OVQXUY3WQ6,0.2,0.55065,0.000000,1.0
A1GQWLAZQZP5VR,A1GQWLAZQZP5VR,0.0,NaN,0.516398,1.0
A1Y3JL53XAO7QV,A1Y3JL53XAO7QV,0.0,NaN,0.836660,1.0
A29GU10CS7PZ1I,A29GU10CS7PZ1I,0.0,NaN,0.408248,1.0
A2P49UVZY1KWAQ,A2P49UVZY1KWAQ,0.2,2.85270,0.547723,1.0
A4IS3ES18TP1C,A4IS3ES18TP1C,0.0,NaN,0.547723,1.0
AOYDGVOZUIHII,AOYDGVOZUIHII,0.2,2.22280,0.000000,1.0
AULLJBJ252TKH,AULLJBJ252TKH,0.2,1.81890,0.547723,1.0


In [42]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,demo_rating_var,in_encoding_workers_list


In [43]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,10,"[A16BUREYCTMSOC, A18GEAFI5MMW02, A1B8OVQXUY3WQ..."
qualified in workers list,47,"[A105M4JOZSR7E6, A151XA53WYXVD7, A1969Q0R4Y0E3..."
qualified not in workers list,7,"[A1FN28N3CFL552, A1T8VILAKHKGR1, A2646YGRS0G84..."


In [44]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A105M4JOZSR7E6', 'A151XA53WYXVD7', 'A1969Q0R4Y0E3J', 'A1DDWUJ5R357SW', 'A1HLQV5HX3WUK4', 'A1IMLTXT2RBEFO', 'A1SL10V8CDNI9H', 'A1VJ4ZOXP9PUCP', 'A1Y8FC2FCI0RZ5', 'A20ZFQ93DF0WA', 'A237PUN7791D62', 'A2541C8MY0BYV3', 'A25PFSORDO3SWQ', 'A25TAR286DACXU', 'A2CZ5PS567SA10', 'A2IC5B7J5YKNOX', 'A2J4GCDXLDHYXX', 'A2JEO9327UXAQ4', 'A2LAXPTVI3IP1B', 'A2LI1OIR4BVJRW', 'A2P9XYDUCQBXWD', 'A2STRYBSFU7KCY', 'A2TQ9IHDHCSQC8', 'A2UWEAIX8UZXAQ', 'A30C7BD86P06O6', 'A31LNMK1LGOIIA', 'A32X8WXMIV4KE8', 'A34S4NIN4XOXWI', 'A39THQVCDUYC1V', 'A39XW0561NNM5C', 'A3ATB5GC4BQIH0', 'A3AU8PZ6HHBCVT', 'A3BDMR7VKG8GRZ', 'A3D8YUG5D5I4ER', 'A3DLP5ADP74EI0', 'A3LBMZYPY37NRW', 'A3RK5N4X7J4JGK', 'A3SMMI1Z1558QE', 'A3VIHILVE7WLKR', 'A7RFFVAR7SANB', 'AC01SVLHOB6HN', 'AF6DVGK1GTFER', 'AIEGSXYADFGQI', 'AJT7C07A187KA', 'ANX6Q4NMZL8EI', 'AOWTBF2L04ENM', 'AW07POT8ABE05']

updating qualifications for participants elligable for test session that are not in the wor

,Worker ID,UPDATE-completed encoding multicategory memory test,UPDATE-completed multicategory memory test
4,A105M4JOZSR7E6,1.0,NaN
113,A151XA53WYXVD7,1.0,NaN
191,A1969Q0R4Y0E3J,1.0,NaN
283,A1DDWUJ5R357SW,1.0,NaN
374,A1HLQV5HX3WUK4,1.0,NaN
394,A1IMLTXT2RBEFO,1.0,NaN
602,A1SL10V8CDNI9H,1.0,NaN
678,A1VJ4ZOXP9PUCP,1.0,NaN
737,A1Y8FC2FCI0RZ5,1.0,NaN
793,A20ZFQ93DF0WA,1.0,NaN


In [45]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(PATH_TO_BATCH/'Batch_workers_for_upload_after_encoding.csv',index=False)

In [46]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 

max_csv_number=99
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 99:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
['AYCKOWD272WLJ', 'A3W4XAY1Z9HV2F', 'A1B973PG17H746', 'ACJTRGTOYBUP0', 'A3QV1YTGI3J6WI', 'A1SRSP0X39CGRW', 'A2G4E4KLMHG03S', 'A1MSG3V72SBKVU', 'AHZTJ2YOJTJ63', 'ARPLYNAONLY7Y', 'A3785TR8D5I81Q', 'A10GMYR513JU0Q', 'AFRA8WYVL1DJ7', 'A3VL1CBZ3BGQK7', 'A141BEGMEHN31L', 'A2K9NQ09JIHPTD', 'A3BN69Q9D7RQGR', 'AXA1G5JO6F0JU', 'A96TDNBSXG3NJ', 'A2BW2A54Q6FVJS', 'A1PJHJB6MD7G3M', 'A2P97OS4ZYS35W', 'A2FQVRBXJZQAWT', 'AQN32EZB7OQRG', 'AU94OKOXHOGXE', 'A39WZD2C8BPP01', 'A179HCVHIA02KX', 'A2PEY75Y7UQSIK', 'A2B3MLFP9BWZ2N', 'A3EHQSJ3XN0PQU', 'AHKAUCJO3UDU5', 'ASAMJC3EOZ9XE', 'A13J80VL8PZNLO', 'A21YNTPZSVT4QX']


In [47]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

A179HCVHIA02KX did not have an encoding file - is not in the shelf
A2893LTGSDN0SM did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
AD9UZ3XGESASI did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A2PC40UQTAFJCM did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A2HKVA03AVZRU2 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
AXA1G5JO6F0JU did not have an encoding file - is not in the shelf
A2PEY75Y7UQSIK did not have an encoding file - is not in the shelf
A2HBCXAK9A7A8D did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
AB6L60HXUNORT did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
A3HFB277V5OXY0 did not have an encoding file - shelf entry changed from: [32, 1683126378555] to [999, 1683126378555]
A21YNTPZSVT4QX did not have an encodi

In [48]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A29167TQYCEXWM absent from WORKERS LIST  - shelf entry changed from: [7, 1683125601961] to [999, 1683125601961]
A15EHYMEITNBWE absent from WORKERS LIST  - shelf entry changed from: [11, 1683125702046] to [999, 1683125702046]
A12PR5DPEZ5I3U absent from WORKERS LIST  - shelf entry changed from: [22, 1683126090054] to [999, 1683126090054]
A7ZTU5G23PGDQ absent from WORKERS LIST  - shelf entry changed from: [38, 1683126687436] to [999, 1683126687436]
A436DJF5TJ5V4 absent from WORKERS LIST  - shelf entry changed from: [49, 1683127369576, 1683127459750, 1683133370749] to [999, 1683127369576, 1683127459750, 1683133370749]
A3UDJL809LPCVW absent from WORKERS LIST  - shelf entry changed from: [53, 1683130329734] to [999, 1683130329734]
ASS3WREGCSW absent from WORKERS LIST  - shelf entry changed from: [72, 1683134401073] to [999, 1683134401073]
A156M0MAQ4T1NC absent from WORKERS LIST  - shelf entry changed from: [77, 1683136725955, 1683136982311, 1683138760525] to [999, 1683136725955, 168313698231

In [49]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A19PB6VUP0RAC3,A19PB6VUP0RAC3,9
A1ABRVTR49OA73,A1ABRVTR49OA73,21
A1AMGHYG5PT0L2,A1AMGHYG5PT0L2,39
A1BQLX0A4W8A8,A1BQLX0A4W8A8,83
A1DIYFGQT1QDV2,A1DIYFGQT1QDV2,31
...,...,...
A25PFSORDO3SWQ,A25PFSORDO3SWQ,99
A1SL10V8CDNI9H,A1SL10V8CDNI9H,11
A1Y8FC2FCI0RZ5,A1Y8FC2FCI0RZ5,17
A3SMMI1Z1558QE,A3SMMI1Z1558QE,24


In [50]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,demo_rating_var,in_encoding_workers_list
A19PB6VUP0RAC3,A19PB6VUP0RAC3,9,NaN,NaN,NaN,NaN,NaN
A1ABRVTR49OA73,A1ABRVTR49OA73,21,NaN,NaN,NaN,NaN,NaN
A1AMGHYG5PT0L2,A1AMGHYG5PT0L2,39,NaN,NaN,NaN,NaN,NaN
A1BQLX0A4W8A8,A1BQLX0A4W8A8,83,NaN,NaN,NaN,NaN,NaN
A1DIYFGQT1QDV2,A1DIYFGQT1QDV2,31,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
A25PFSORDO3SWQ,A25PFSORDO3SWQ,99,A25PFSORDO3SWQ,1.0,0.592400,1.471960,1.0
A1SL10V8CDNI9H,A1SL10V8CDNI9H,11,A1SL10V8CDNI9H,1.0,1.055400,0.752773,1.0
A1Y8FC2FCI0RZ5,A1Y8FC2FCI0RZ5,17,A1Y8FC2FCI0RZ5,1.0,1.013040,0.752773,1.0
A3SMMI1Z1558QE,A3SMMI1Z1558QE,24,A3SMMI1Z1558QE,0.8,1.893475,0.632456,1.0


In [51]:
#this is the updated shelf: <<<< copy >>>> from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A19PB6VUP0RAC3": [9, 1682949997166, 1683029612907, 1683030155680], "A1ABRVTR49OA73": [21, 1682950049077, 1683032370299, 1683041181284], "A1AMGHYG5PT0L2": [39, 1682950918763, 1683033162919], "A1BQLX0A4W8A8": [83, 1682964250527, 1683030823745, 1683030930529, 1683031003758, 1683031140415, 1683031305050, 1683039824808, 1683039997462], "A1DIYFGQT1QDV2": [31, 1682950355274, 1683046479268], "A1PRUVH5Q0T9FS": [4, 1682949745590, 1683025740833], "A1Q3GKUJF8EWHG": [93, 1682981721758, 1683056413973, 1683061178454], "A1YALKEEJHHWSO": [62, 1682954227335, 1683032969147, 1683033390032], "A2AN972XU7OY8W": [81, 1682962776227, 1683049184845], "A2F48NO7AD9VKK": [23, 1682950399994, 1683036067950], "A2FREFX5AGACUZ": [46, 1682980285878, 1683054419004, 1683054438957, 1683062649288, 1683065582318], "A2J9NFY45NKR2N": [18, 1682949882285, 1683033522450], "A2NC28XJKAFJV": [78, 1682962022271, 1683046189022], "A39KMTFHRJJASK": [82, 1682963122891, 1683049864717], "A3EWBTHYFI4I6Q": [48, 1682951641562, 1683013763685